In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup


In [ ]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, Trainer

from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, Trainer

import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, Trainer

import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler
## **GPT-2 with feature: title, structure**
idea comes from : https://colab.research.google.com/drive/13dZVYEOMhXhkXWfvSMVM1TTtUDrT6Aeh?usp=sharing#scrollTo=vCPohrZ-CTWu

In [ ]:
df= pd.read_csv('/kaggle/input/630-final/wn21630project_input_processed.csv')
df.fillna(' ',inplace = True)


with open('./train_dataset.txt','w') as f:
    f.write(''.join(d[:int(length*0.6)]))
with open('./dev_dataset.txt','w') as f:
    f.write(' <Newsong> '.join(d[int(length*0.6):int(length*0.8)]))
with open('./test_dataset.txt','w') as f:
    f.write(' <Newsong> '.join(d[int(length*0.8):]))
train_path = './train_dataset.txt'
dev_path = './dev_dataset.txt'

In [ ]:
SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",                    
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}

In [ ]:
keywords = ['intro', 'verse', 'chorus', 'bridge', 'hook']
input = []
for index,row in df.iterrows():
    key_i, text_i = [],[]
    for i in keywords:
        if row[i] !=' ':
            key_i.append(i)
            text_i.append(re.sub('(\n)+','. ',row[i]))
    input_i = SPECIAL_TOKENS['bos_token'] + row.title + \
        SPECIAL_TOKENS['sep_token'] + ', '.join(key_i) + SPECIAL_TOKENS['sep_token'] + \
        '. '.join(text_i) + SPECIAL_TOKENS['eos_token']
    input.append(input_i)

In [ ]:
length = len(input)

In [ ]:
with open('./train_dataset.txt','w') as f:
    f.write(' <Newsong> '.join(input[:int(length*0.6)]))
with open('./dev_dataset.txt','w') as f:
    f.write(' <Newsong> '.join(input[int(length*0.6):int(length*0.8)]))
with open('./test_dataset.txt','w') as f:
    f.write(' <Newsong> '.join(input[int(length*0.8):]))

In [ ]:
train_path = './train_dataset.txt'
dev_path = './dev_dataset.txt'

In [ ]:
class myDataset(Dataset):

    def __init__(self, data, tokenizer):
        keywords = ['intro', 'verse', 'chorus', 'bridge', 'hook']
        input = []
        for index,row in data.iterrows():
            key_i, text_i = [],[]
            for i in keywords:
                if row[i] !=' ':
                    key_i.append(i)
                    text_i.append(re.sub('(\n)+',' <Newline> ',row[i]))
            input_i = SPECIAL_TOKENS['bos_token'] + row.title + \
                SPECIAL_TOKENS['sep_token'] + ', '.join(key_i) + SPECIAL_TOKENS['sep_token'] + \
                ' <Newsong> '.join(text_i) + SPECIAL_TOKENS['eos_token']
            input.append(input_i)
 
        
        self.tokenizer = tokenizer 
        self.text      = input

    #---------------------------------------------#

    def __len__(self):
        return len(self.text)

    #---------------------------------------------#
    
    def __getitem__(self,i):
        encodings_dict = tokenizer(self.text[i],                                   
                                   truncation=True, 
                                   max_length=MAXLEN, 
                                   padding="max_length")   
        
        input_ids = encodings_dict['input_ids']
        attention_mask = encodings_dict['attention_mask']
        
        return {'label': torch.tensor(input_ids),
                'input_ids': torch.tensor(input_ids), 
                'attention_mask': torch.tensor(attention_mask)}

In [ ]:
training_size = 1070
deving_size = 357
testing_size = 358
size = [training_size, deving_size,testing_size]

In [ ]:
DEBUG           = False

INPUT_DIR       = 'articles'

USE_APEX        = True
APEX_OPT_LEVEL  = 'O1'

MODEL           = 'gpt2' #{gpt2, gpt2-medium, gpt2-large, gpt2-xl}

UNFREEZE_LAST_N = 6 #The last N layers to unfreeze for training


                    
MAXLEN          = 768  #{768, 1024, 1280, 1600}

TRAIN_SIZE      = 0.8

if USE_APEX:
    TRAIN_BATCHSIZE = 4
    BATCH_UPDATE    = 16
else:
    TRAIN_BATCHSIZE = 2
    BATCH_UPDATE    = 32

EPOCHS          = 4
LR              = 5e-4
EPS             = 1e-8
WARMUP_STEPS    = 1e2

SEED            = 2020

In [ ]:
def get_tokenier(special_tokens=None):
    tokenizer = AutoTokenizer.from_pretrained(MODEL) #GPT2Tokenizer

    if special_tokens:
        tokenizer.add_special_tokens(special_tokens)
        print("Special tokens added")
    return tokenizer

def get_model(tokenizer, special_tokens=None, load_model_path=None):

    #GPT2LMHeadModel
    if special_tokens:
        config = AutoConfig.from_pretrained(MODEL, 
                                            bos_token_id=tokenizer.bos_token_id,
                                            eos_token_id=tokenizer.eos_token_id,
                                            sep_token_id=tokenizer.sep_token_id,
                                            pad_token_id=tokenizer.pad_token_id,
                                            output_hidden_states=False)
    else: 
        config = AutoConfig.from_pretrained(MODEL,                                     
                                            pad_token_id=tokenizer.eos_token_id,
                                            output_hidden_states=False)    

    #----------------------------------------------------------------#
    model = AutoModelForPreTraining.from_pretrained(MODEL, config=config)

    if special_tokens:
        #Special tokens added, model needs to be resized accordingly
        model.resize_token_embeddings(len(tokenizer))

    if load_model_path:
        model.load_state_dict(torch.load(load_model_path))

    model.cuda()
    return model

In [ ]:
%%time

tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                #   load_model_path='pytorch_model.bin'
                 )

In [ ]:
train_data = df.iloc[:training_size]
val_data = df.iloc[training_size:(deving_size+training_size)]

train_dataset = myDataset(train_data, tokenizer)
val_dataset = myDataset(val_data, tokenizer)


In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(dev_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=32)
        #   block_size=128)
     
    dev_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=dev_path,
          block_size=32) 
        #   block_size=128)  
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,dev_dataset,data_collator

train_dataset,dev_dataset,data_collator = load_dataset(train_path,dev_path,tokenizer)

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import GPT2LMHeadModel
#model = GPT2LMHeadModel.from_pretrained('gpt2')


training_args = TrainingArguments(
    output_dir="./output", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=4, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    # per_device_train_batch_size=16, # batch size for training
    # per_device_eval_batch_size=32,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

In [ ]:
trainer.train()

from transformers import TextDataset,DataCollatorForLanguageModeling
data = df
text, keylist, title = [],[],[]
keywords = ['intro', 'verse', 'chorus', 'bridge', 'hook']

for index, row in data.iterrows():
    title.append(row.title)
    key_i = []
    text_i = ''
    for key in keywords:
        if row[key] !=' ':
            if key == 'hook': key= 'chorus' 
            key_i.append(key)
            #print(index)
            text_i +=re.sub('(\n)+', ' <Newline> ',row[key])
    text.append(text_i)
    keylist.append(",".join(key_i))
d = []
for i in range(0,len(data)):
    d.append('<|endoftext|>' + title[i] + \
                '<|endoftext|>'+ keylist[i] + '<|endoftext|>'+ \
                text[i] + '<|endoftext|>')
        


length = len(d)
train_data = ' <Newsong> '.join(d[:int(length*0.6)])
dev_data = ' <Newsong> '.join(d[int(length*0.6):int(length*0.8)])
test_data = ' <Newsong> '.join(d[int(length*0.8):])

with open('./train_dataset.txt','w') as f:
    f.write(train_data)

with open('./dev_dataset.txt','w') as f:
    f.write(dev_data)

train_path = './train_dataset.txt'
dev_path = './dev_dataset.txt'

In [ ]:
title = "We don't talk anymore"
#kw = myDataset.join_keywords(keywords, randomize=False)

prompt =  SPECIAL_TOKENS['bos_token'] + title +  SPECIAL_TOKENS['sep_token'] + 'intro, verse, chorus' +SPECIAL_TOKENS['sep_token']  
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval();

In [ ]:
sample_outputs = model.generate(generated, 
                                do_sample=True,   
                                min_length=300, 
                                max_length=500,
                                top_k=30,                                 
                                top_p=0.7,        
                                temperature=0.9,
                                repetition_penalty=2.0,
                                num_return_sequences=1
                                )

for i, sample_output in enumerate(sample_outputs):
    text = tokenizer.decode(sample_output, skip_special_tokens=False)
    #a = len(title) + len(','.join(keywords))    
    print("{}: {}\n\n".format(i+1,  text))

In [ ]:
output = []
for index, row in df.iloc[(deving_size+training_size):].iterrows():
    prompt =  SPECIAL_TOKENS['bos_token'] + row.title +  SPECIAL_TOKENS['sep_token'] + 'intro, verse, chorus' +SPECIAL_TOKENS['sep_token']           
    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
    device = torch.device("cuda")
    generated = generated.to(device)

    sample_outputs = model.generate(generated, 
                                do_sample=True,   
                                min_length=400, 
                                max_length=500,
                                top_k=30,                                 
                                top_p=0.7,        
                                temperature=0.9,
                                repetition_penalty=2.0,
                                num_return_sequences=1
                                )
    text = tokenizer.decode(sample_outputs[0], skip_special_tokens=False)
    output.append(text)


In [ ]:
import json
new_dict = {'output_song_with_structure':output}
with open("./output_song_with_structure.json","w") as f:
    json.dump(new_dict,f)
    print("加载入文件完成...")